Data Mining: Basic Concepts - Winter 2023/24
---------------
``` 
> University of Konstanz 
> Department of Computer and Information Science
> Maximilian T. Fischer, Frederik Dennig, Yannick Metz, Udo Schlegel
```
__Organize in teams of 2 people, return the exercise on time using ILIAS__

---

Assignment 08 in Python 
---------------
- ___Please put your names and student IDs here___:
    - ___Please put your names and student IDs here___:
    - _Lorenz Rückert_, _01/911915_
    - _Lennart Kasserra_, _01/1358216_

---

#### Excercise 1: Lazy vs Eager: Theory

**a) Explain the difference between an eager learner and a lazy learner.**

While an eager learner learns an abstract model based on a training set, which it uses to classify new observations, lazy learners store data tuples and classify newly encountered tuples based on the already stored data tuples.

**b) Provide an example of a machine learning algorithm that is an eager learner and one that is a lazy learner.**

Let's assume a classification context. Decision Trees would be an example of an eager learner - first they are trained on a training set, forming decision rules. Newly encountered observations are then classified based on these learned decision rules.

An example of a lazy learner would be k-Nearest Neighbor (kNN) classification: upon encountering a new tuple, the k known tuples with the shortest distance to the new tuple are detected & their most common class is assigned to the new tuple.

*You are given a dataset with 1 million rows and 10 columns. You need to build a machine learning model to predict a binary outcome based on the values in the columns. You have two options: a decision tree or a k-nearest neighbors algorithm.*
  
**b) Which algorithm should you choose and why? Explain your reasoning in detail, taking into account the size and complexity of the dataset and the computational resources available to you.**

I would use a decision tree. Decision trees are inexpensive to construct, which is advantageous given the size and complexity of the data. It would be computationally expensive to run a kNN-query on 1 million rows and 10 columns for every input. A decision tree would also be advantageous in terms of interpretability, as kNN does not generate any explicit knowledge about the classes (although interpretability would probably, to some point, be impaired by the complexity of the data set).

#### Excercise 2: Lazy vs Eager: Practical

In this exercise, we want to implement a basic k-nearest neighbor algorithm.   
For this, we need a similarity function and a general k-nearest neighbor function.  
We will use the breast cancer dataset we previously used for the SVM with an 80/20 train test split.    
  
**Please implement the k-nearest neighbor algorithm on your own and show that the implementation works for the breast cancer data.**  
_(Hint: use the `sklearn.model_selection.train_test_split` method and the parameter `random_state=0`)_

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df = (df - df.min()) / (df.max() - df.min())
df['diagnosis'] = data.target

features = list(data.feature_names)

train, test = train_test_split(df, test_size=0.2, random_state=0)

In [2]:
# Performance is bad, but it is made with passion:

class KNNClassifier:

    predictions = []

    def __init__(self, k: int) -> None:
        self.k = k

    def train(self, train_features: pd.DataFrame, train_target: pd.Series) -> None:
        self.train_features = train_features
        self.train_target = train_target

    def __euclidean_dist(self, new_tup, known_tup):
        return np.sqrt(sum((new_tup - known_tup)**2))
    
    def predict(self, test_features: pd.DataFrame) -> list:
        
        predictions = []

        for _, row in test_features.iterrows():
            # find closest rows:
            distances = [self.__euclidean_dist(row, xrow) for _, xrow in self.train_features.iterrows()]
            ids = sorted(range(len(distances)), key=lambda sub: distances[sub])[:self.k]
            # find their classes & store most frequent one:
            values = self.train_target.iloc[ids]
            prediction = values.mode()[0]
            predictions.append(prediction)


        self.predictions = predictions
        return predictions
    
    def evaluate(self, test_target: pd.Series) -> float:
        return sum(self.predictions == test_target) / len(test_target)

In [8]:
# Just showing it works with some k (albeit very slowly):
knn = KNNClassifier(k=5)
knn.train(train[features], train["diagnosis"])
knn.predict(test[features]) # predictions are stored in knn.predictions...
knn.evaluate(test["diagnosis"])

0.9649122807017544

#### Excercise 3: Evaluation Metrics Theory

**a) Explain why different evaluation metrics are used for evaluating the performance of classifiers.**

Different metrics reflect different tradeoffs and have different strengths. For example, ROC curves are better fit for when observations are balanced between classes, while Precision & Recall are more appropriate for imbalanced data sets.

**b) Provide examples of common evaluation metrics for classification algorithms, and explain the strengths and limitations of each metric in the context of classifier evaluation.**

* **Precision & Recall:** *Precision* represents the ability of a system to present only relevant items, while *Recall* is a measure of the ability of a system to present all relevant items. They represent the tradeoff between the true positive rate and the positive predictive value of a model.

* **ROC:** *ROC* (Receiver Operating Characteristic) summarises the tradeoff between the true positive rate, and the false positive rate for a model.

While ROC curves are better fit for when observations are balanced between classes, Precision & Recall are appropriate for imbalanced data sets.

**c) Explain how a perfect precision-recall curve looks like.**

A perfect precision-recall curve maximizes both precision and recall; we would like the curve to stay as close to a value of 1 as possible for both of these; in the conventional way of visualizing, we would expect the curve to pivot towards the top right.

#### Excercise 4: Evaluation Metrics Practical

After thinking about the different evaluation metrics, we want to see them in real-world scenarios and inspect their workings.  
We tackle two datasets for this task and use the machine learning algorithms we have already explored.  
The first dataset will be the breast cancer dataset again. The second one will be the cover-type forest dataset.  
Both datasets have unique properties and need special care. Handling different data is not always easy and selecting proper algorithms is even more challenging.
Thus, we focus on the different evaluation metrics to select one good working algorithm.  
In this case, we will compare the sci-kit learn implementations of Naive Bayes, Decision Trees, SVMs, and k-Nearest Neighbors based on the classification accuracy and F1-score.  
  
As the cover type data has more than two classes, we will only use the second and third class samples.  
  
**Please implement a comparison between the sci-kit learn implementations of Naive Bayes, Decision Trees, SVMs, and k-Nearest Neighbors on the breast cancer data and cover type data for your own implementation of accuracy and F1-score.**    
_(Hint: use the `sklearn.model_selection.train_test_split` method and the parameter `random_state=0`)_  
_(Hint: use the `sklearn.naive_bayes.GaussianNB`, `sklearn.tree.DecisionTreeClassifier`, `sklearn.svm.SVC`, `sklearn.neighbors.KNeighborsClassifier` methods with default parameters)_  

*First we do it for the forest cover:*

In [10]:
from sklearn.datasets import fetch_covtype
data = fetch_covtype()

target = data['target'][np.logical_or(data['target'] == 2, data['target'] == 3)] - 2
data = data['data'][np.logical_or(data['target'] == 2, data['target'] == 3)]

In [13]:
train_features, test_features = train_test_split(data, test_size=0.2, random_state=0)
train_target, test_target = train_test_split(target, test_size=0.2, random_state=0)

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

naive_bayes = GaussianNB()
decision_tree = DecisionTreeClassifier()
svm = SVC()
knn = KNeighborsClassifier()

for model in [naive_bayes, decision_tree, svm, knn]:
    model.fit(train_features, train_target)

In [16]:
predictions = {
    "naive_bayes":   naive_bayes.predict(test_features),
    "decision_tree": decision_tree.predict(test_features),
    "svm":           svm.predict(test_features),
    "knn":           knn.predict(test_features)
}

In [17]:
from sklearn.metrics import confusion_matrix

def get_metrics(actual, predicted):
    cm = confusion_matrix(actual, predicted)
    TP, FP, FN, TN = cm.ravel()
    accuracy = (TP + TN) / len(actual)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (precision * recall) / (precision + recall)
    return {"accuracy": accuracy, "f1": f1}

In [24]:
all_model_metrics = []
for model, pred in predictions.items():
    metrics = get_metrics(test_target, pred)
    all_model_metrics.append({"model": model, "accuracy": metrics["accuracy"], "f1": metrics["f1"]})

metrics_forest = pd.DataFrame(all_model_metrics)
metrics_forest

,model,accuracy,f1
0,naive_bayes,0.915203,0.949890
1,decision_tree,0.995017,0.997193
2,svm,0.964285,0.980074
3,knn,0.997258,0.998455


Here, kNN has the highest accuracy, and the highest f1-score, closely followed by the decision tree.

*Next, we jump to the breast cancer dataset:*

In [28]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df = (df - df.min()) / (df.max() - df.min())
df['diagnosis'] = data.target
target = "diagnosis"

features = list(data.feature_names)

train, test = train_test_split(df, test_size=0.2, random_state=0)

In [38]:
naive_bayes2 = GaussianNB()
decision_tree2 = DecisionTreeClassifier()
svm2 = SVC()
knn2 = KNeighborsClassifier()

for model in [naive_bayes2, decision_tree2, svm2]:
    model.fit(train[features], train[target])

# kNN needs special treatment when dealing with pandas objects
knn2.fit(train[features].values, train[target].values)

KNeighborsClassifier()

In [39]:
predictions = {
    "naive_bayes":   naive_bayes2.predict(test[features]),
    "decision_tree": decision_tree2.predict(test[features]),
    "svm":           svm2.predict(test[features]),
    "knn":           knn2.predict(test[features].values)
}

In [42]:
all_model_metrics = []
for model, pred in predictions.items():
    metrics = get_metrics(test[target], pred)
    all_model_metrics.append({"model": model, "accuracy": metrics["accuracy"], "f1": metrics["f1"]})

metrics_cancer = pd.DataFrame(all_model_metrics)
metrics_cancer.sort_values("accuracy")

,model,accuracy,f1
0,naive_bayes,0.903509,0.884211
1,decision_tree,0.903509,0.888889
3,knn,0.964912,0.955556
2,svm,0.973684,0.967742


This time, the SVM outperforms the other models.